In [2]:
import pandas as pd
import numpy as np
import itertools
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from functions.graph_with_prediction import graph_with_prediction

import warnings 
warnings.simplefilter('ignore')

In [3]:
train_oil_year_accum_without_gaps = pd.read_csv('../../data/Преобразованные_данные/train_oil_year_accum_without_gaps.csv')
train_oil_year_accum_without_gaps.head()

,MEASURED_IN_DATE,WELL_NAME,CHARWORK,WATER_CUT,P_ZAB,INTAKE,WELLHEAD_PRESSURE,LIQ_RATE,OIL_RATE,WATER_RATE,Q_LIQ_ACCUM,Q_OIL_ACCUM,Q_WATER_ACCUM
0,2020-08-31,1,НЕФ,60.0,52.57,0.0,NaN,247.0,98.80,148.20,32268.0,18217.47,14050.53
1,2020-09-01,1,НЕФ,60.0,52.57,0.0,NaN,244.0,97.60,146.40,32512.0,18315.07,14196.93
2,2020-09-02,1,НЕФ,59.0,51.55,0.0,NaN,243.0,99.63,143.37,32755.0,18414.70,14340.30
3,2020-09-03,1,НЕФ,60.0,51.57,0.0,NaN,241.0,96.40,144.60,32996.0,18511.10,14484.90
4,2020-09-04,1,НЕФ,62.0,51.60,0.0,NaN,240.0,91.20,148.80,33236.0,18602.30,14633.70


In [4]:
data = train_oil_year_accum_without_gaps[train_oil_year_accum_without_gaps['WELL_NAME'] == 1][['MEASURED_IN_DATE', 'OIL_RATE']]
data.set_index('MEASURED_IN_DATE', drop=True, inplace=True)

total_train  = int(0.7 * data.shape[0])

y_train, y_test = data[: total_train], data[total_train :]

In [ ]:
arima = ARIMA(y_train, order=(0, 0, 0), freq='D')
p_values = range(3, 5)
q_values = range(3, 5)
d_values = range(3, 5)
params = {'p': p_values,
          'd': d_values,
          'q': q_values}

pdq = list(itertools.product(p_values, q_values, d_values))
seasonal_PDQ = [(x[0], x[1], x[2], 7) for x in pdq]

best_aic = np.inf
best_params = None
best_seasonal_params = None

for params in pdq:
    for seasonal_params in seasonal_PDQ:
        try:
            model = ARIMA(y_train, order=params, seasonal_order=seasonal_params)
            result = model.fit()
            if result < best_aic:
                best_aic = result
                best_params = params
                best_seasonal_params = seasonal_params
        except:
            continue
model = ARIMA(y_train, order=best_params, seasonal_order=best_seasonal_params)
model.fit()

In [ ]:
graph_with_prediction(y_train=y_train['OIL_RATE'], 
                      y_test=y_test['OIL_RATE'],
                      answer_model=forecast,
                      model_name='ARIMA')